# SQL and DataFrames: Hands-on Exercises (Python)

This notebook contains hands-on exercises used in conjunction with the DataFrames module. Each section corresponds to a section in the lecture. Your instructor will tell you when it's time to do each section of this notebook.

## Schema Inference

In this exercise, let's explore schema inference. We're going to be using a file called `people.txt`. The data is structured, but it has no self-describing schema. And, it's not JSON, so Spark can't infer the schema automatically. Let's create an RDD and look at the first few rows of the file.

In [3]:
rdd = sc.textFile("dbfs:/mnt/databricks-corp-training/common/dataframes/people.txt")
for line in rdd.take(10):
  print line

As you can see, each line consists of the same information about a person:

* first name
* middle name
* last name
* gender ("M" or "F")
* birth date, in `yyyy-mm-dd` form
* a salary
* a United States Social Security Number

(Before you get _too_ excited and run out to apply for a bunch of credit cards, the Social Security Numbers are all fake.)

Clearly, the file has a schema, but Spark can't figure out what it is.

Read through the following code to see how we can apply a schema to the file. Then, run it, and see what happens.

In [5]:
from datetime import datetime
from collections import namedtuple

Person = namedtuple('Person', ['first_name', 'middle_name', 'last_name', 'gender', 'birth_date', 'salary', 'ssn'])

def map_to_person(line):
  cols = line.split(":")
  return Person(first_name  = cols[0],
                middle_name = cols[1],
                last_name   = cols[2],
                gender      = cols[3],
                birth_date  = datetime.strptime(cols[4], "%Y-%m-%d"),
                salary      = int(cols[5]),
                ssn         = cols[6])
    
people_rdd = rdd.map(map_to_person)
df = people_rdd.toDF()


**Question:** What could go wrong in the above code? How would you fix the problems?

Now, let's sample some of the data.

In [8]:
sampledDF = df.sample(withReplacement = False, fraction = 0.02, seed = 1887348908234L)
display(sampledDF)

Finally, let's run a couple SQL commands.

In [10]:
df.registerTempTable("people")

In [11]:
%sql SELECT * FROM people WHERE birth_date >= '1970-01-01' AND birth_date <= '1979-12-31' ORDER BY birth_date, salary

In [12]:
%sql SELECT concat(first_name, " ", last_name) AS name, gender, year(birth_date) AS birth_year, salary FROM people WHERE salary < 50000

### ![](http://i.imgur.com/RdABwEB.png) STOP HERE.

Let's switch back to the slides.

## select and filter (and a couple more)

We've now seen `printSchema()`, `show()`, `select()` and `filter()`. Let's take them for a test drive.

First, let's look at the schema.

In [15]:
df.printSchema()

Now, let's look at `show()`.

In [17]:
df.show() # show the first 20 rows of the DataFrame

`show()` is a good way to get a quick feel for your data. Of course, in a Databricks notebook, the `display()` helper is better. However, if you're using `spark-shell`, the `display()` helper isn't available.

In [19]:
display(df)

Let's look at `select()`. Run the following cell. What does it return?

In [21]:
df.select(df["first_name"], df["last_name"], df["gender"])

**Remember**: Transformations are _lazy_. The `select()` method is a transformation.

All right. Let's look at result of a `select()` call.

In [23]:
df.select(df["first_name"], df["last_name"], df["gender"]).show(10)

Finally, let's take a look at `filter()`, which can be used to filter data _out_ of a data set.

**Question**: What the does the following code actually do?

In [25]:
df.filter(df["gender"] == "M")

If you're familiar with the DataFrames Scala API, you'll notice that we use double-equals (`==`) in that comparison, not the triple-equals (`===`) we use in Scala. If you switch between languages, keep that in mind.

`filter()`, like `select()`, is a transformation: It's _lazy_.

Let's try something a little more complicated. Let's combine two `filter()` operations with a `select()`, displaying the results.

In [27]:
df2 = df.filter(df["gender"] == "M").filter(df["salary"] > 100000).select(df["first_name"], df["last_name"], df["salary"])
display(df2)

### ![](http://i.imgur.com/RdABwEB.png) STOP HERE.

Let's switch back to the slides.

## orderBy, groupBy and alias

Up in the first section of this notebook, we ran this SQL statement:

```
SELECT * FROM people WHERE birthDate >= '1970-01-01' AND birthDate <= '1979-12-31' ORDER BY birthDate, salary
```

Let's try that same query with the programmatic DataFrames API.

In [31]:
display(
  df.filter(df["birth_date"] >= "1970-01-01").filter(df["birth_date"] <= "1979-12-31").orderBy(df.birth_date, df.salary)
)

There are several things to note.

1. We did not have to convert the date literals ("1970-01-01" and "1979-12-31") into `datetime` objects before using them in the comparisons.
2. We used two different ways to specify the columns: `df["firstName"]` and `df.first_name`.

Let's try a `groupBy()` next.

In [33]:
display( df.groupBy(df["salary"]) )

Okay, that didn't work. Note that `groupBy()` returns something of type `GroupedData`, instead of a `DataFrame`. There are other methods on `GroupedData` that will convert back to a DataFrame. A useful one is `count()`.

**WARNING**: Don't confuse `GroupedData.count()` with `DataFrame.count()`. `GroupedData.count()` is _not_ an action. `DataFrame.count()` _is_ an action.

In [35]:
x = df.groupBy(df["salary"]).count()  # What is x?

In [36]:
display(x)

Let's add a filter and, while we're at it, rename the `count` column.

In [38]:
display( x.filter(x['count'] > 1).select("salary", x["count"].alias("total")) )